# core

> Fill in a module description here

## Setup

In [1]:
#| default_exp core

In [2]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *
from IPython.display import HTML

from pyngrok import ngrok

# for magic
from IPython import get_ipython
from IPython.core.magic import register_cell_magic
import ast

## Headers
> css, javascripts

### Bootstrap

In [3]:
#| export
def bootstrap_hdrs(include_icon=True):
    css = Link(href='https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css', rel='stylesheet')
    js = Script(src='https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js')
    icon = Link(href='https://cdn.jsdelivr.net/npm/bootstrap-icons@1.11.3/font/bootstrap-icons.min.css', rel='stylesheet')
    return [css,js,icon] if include_icon else [css,js]

### Datatables for dynamic table

In [4]:
#| export
def datatable_hdrs():
    dt_s1 = Script(src='https://code.jquery.com/jquery-3.7.1.js')
    dt_s2 = Script(src='https://cdn.datatables.net/2.1.5/js/dataTables.js')
    dt_s3=Script(src='https://cdn.datatables.net/2.1.5/js/dataTables.bootstrap5.js')
    dt_c1 = Link(rel='stylesheet',href='https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/5.3.0/css/bootstrap.min.css')
    dt_c2 = Link(rel='stylesheet',href='https://cdn.datatables.net/2.1.5/css/dataTables.bootstrap5.css')
    return [dt_s1,dt_s2,dt_s3,dt_c1,dt_c2]

### Conditional Pico

In [5]:
#| export
def cond_pico_hdrs():
    conditional_pico = Link(rel='stylesheet', href='https://cdn.jsdelivr.net/npm/@picocss/pico@2/css/pico.conditional.min.css')
    return [conditional_pico]

### Copy TextArea js

In [6]:
#| export
def copy_js():
    script_copy_button=Script(src="./imports/copy_button.js")
    return [script_copy_button]

### Autocomplete js

In [ ]:
#| export
def autocomplete_js():
    jquery = Script(src='https://code.jquery.com/jquery-3.6.0.min.js')
    script_autocomplete = Script(src='./imports/autocomplete.js')
    return [jquery,script_autocomplete]

### Download csv/svg js

In [8]:
#| export
def download_js():
    download_script = Script(src='./imports/download_csv_svg.js')
    return [download_script]

## show() in cell magic

Fasthtml updates in the latest version that in jupyter notebook, no need to use show() when importing fasthtml.jupyter

In [4]:
#| export
@register_cell_magic
def s(line, cell):
    # Parse the line argument to check for a 'debug' flag
    debug = 'debug' in line.lower()
    
    # Parse the cell content into an AST
    tree = ast.parse(cell)
    
    # Collect all expressions
    expressions = [node.value for node in tree.body if isinstance(node, ast.Expr)]
    
    # If there are expressions, wrap them all in a single show() call
    if expressions:
        show_call = ast.Expr(
            ast.Call(
                func=ast.Name(id='show', ctx=ast.Load()),
                args=expressions,
                keywords=[]
            )
        )
        # Replace the original expressions with the single show() call
        tree.body = [node for node in tree.body if not isinstance(node, ast.Expr)] + [show_call]
    
    # Convert the modified AST back to source code
    modified_code = ast.unparse(tree)
    
    # If debug is True, print the modified code
    if debug:
        print("Modified code:")
        print(modified_code)
    
    # Execute the modified code
    get_ipython().run_cell(modified_code)

Use %%s in a cell, so that you don't have to wrap everything in show()

In [5]:
%%s
P('sdf')

## Jupyter live on colab

### Initiate ngrok tunnel
> get public url that connect to localhost:8000

In [6]:
#| export
def start_ngrok(token,port=8000):
    
    # Get token from ngrok.com --> login --> Your Authtoken
    ngrok.set_auth_token(token)
    
    # Start a tunnel on port
    ngrok_tunnel = ngrok.connect(port)
    
    # Return the public URL
    return ngrok_tunnel.public_url

Find ngrok token in https://dashboard.ngrok.com/get-started/your-authtoken

In [9]:
from getpass import getpass

token = getpass("Type your token: ")

Type your token:  ········


In [15]:
url = start_ngrok(token)
url

'https://acbd-18-208-186-214.ngrok-free.app'

### Show Live page in jupyter cell

In [18]:
#| export
def htmx(url,path='',height='auto'):
    "An iframe which displays the HTMX application in a notebook."
    return HTML(f'<iframe src="{url}{str(path)}" style="width: 100%; height: {height}; border: none;" ' + """onload="{
        let frame = this;
        window.addEventListener('message', function(e) {
            if (e.data.height) frame.style.height = (e.data.height+1) + 'px';
        }, false);
    }" allow="accelerometer; autoplay; camera; clipboard-read; clipboard-write; display-capture; encrypted-media; fullscreen; gamepad; geolocation; gyroscope; hid; identity-credentials-get; idle-detection; magnetometer; microphone; midi; payment; picture-in-picture; publickey-credentials-get; screen-wake-lock; serial; usb; web-share; xr-spatial-tracking"></iframe> """)

Example with websocket:

In [ ]:
# url = start_ngrok(YOUR_NGROK_TOKEN)
app,rt = fast_app(exts='ws', live=True)
server = JupyUvi(app)
push = Pusher(app)

In [ ]:
@app.get
def click():
    return P('hi')

In [ ]:
push(A('click',hx_get='/click',hx_target='#dest'),Div(id='dest'))

In [ ]:
HTMX_public(url) # click `visit site` for initial page

In [ ]:
server.stop()

Normal example:

In [11]:
# url = start_ngrok(YOUR_NGROK_TOKEN)
app,rt = fast_app(live=True)
server = JupyUvi(app)

In [12]:
@rt
def index():
    return Titled('Hello, Jupyter',
           P('Welcome to the FastHTML + Jupyter example'),
           Button('Click', hx_get='/click', hx_target='#dest'),
           Div(id='dest')
    )

@app.get
def click():
    return P('Hi')

In [19]:
HTMX_public(url) # click `visit site`

In [ ]:
server.stop()

### push() in cell magic

In [ ]:
#| export
@register_cell_magic
def p(line, cell):
    # Parse the line argument to check for a 'debug' flag
    debug = 'debug' in line.lower()
    
    # Parse the cell content into an AST
    tree = ast.parse(cell)
    
    # Collect all expressions
    expressions = [node.value for node in tree.body if isinstance(node, ast.Expr)]
    
    # If there are expressions, wrap them all in a single show() call
    if expressions:
        show_call = ast.Expr(
            ast.Call(
                func=ast.Name(id='push', ctx=ast.Load()),
                args=expressions,
                keywords=[]
            )
        )
        # Replace the original expressions with the single show() call
        tree.body = [node for node in tree.body if not isinstance(node, ast.Expr)] + [show_call]
    
    # Convert the modified AST back to source code
    modified_code = ast.unparse(tree)
    
    # If debug is True, print the modified code
    if debug:
        print("Modified code:")
        print(modified_code)
    
    # Execute the modified code
    get_ipython().run_cell(modified_code)

Instead of wrapping everything in push, use %%p in a cell to do the same thing

In [ ]:
url = start_ngrok(YOUR_NGROK_TOKEN)
app,rt = fast_app(exts='ws', live=True)
server = JupyUvi(app) # JupyUvi is replaced with ws_client and setup_ws in the latest version
push = Pusher(app)

@app.get
def click():
    return P('hi')

In [ ]:
%%p
A('click',hx_get='/click',hx_target='#dest'),Div(id='dest')

In [ ]:
HTMX_public(url)

In [ ]:
server.stop()

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()